# Scrapy
- 웹사이트에서 데이터 수집을 위한 오픈소스 파이썬 프레임워크
- 멀티스레딩으로 데이터 수집
- gmarket 상품데이터 수집

In [1]:
# install scrapy
!pip install scrapy

## 1. make project

In [3]:
import scrapy , requests
from scrapy.http import  TextResponse

In [14]:
!scrapy startproject news

Error: scrapy.cfg already exists in C:\Users\User\Desktop\에이블스쿨\24.09\웹크롤링\웹크롤링_실습자료\news


In [18]:
!tree news /F


폴더 PATH의 목록입니다.
볼륨 일련 번호는 385B-BB36입니다.
C:\USERS\USER\DESKTOP\에이블스쿨\24.09\웹크롤링\웹크롤링_실습자료\NEWS
│  scrapy.cfg
│  
└─news
    │  items.py
    │  middlewares.py
    │  pipelines.py
    │  settings.py
    │  __init__.py
    │  
    └─spiders
            __init__.py
            


- scrapy structure
    - items : 데이터의 모양 정의
    - middewares : 수집할때 header 정보와 같은 내용을 설정
    - pipelines : 데이터를 수집한 후에 코드를 실행
    - settings : robots.txt 규칙, 크롤링 시간 텀등을 설정
    - spiders : 크롤링 절차를 정의

## 2. xpath
- link, contents

In [22]:
import scrapy, requests
from scrapy.http import TextResponse

In [26]:
url= 'https://news.daum.net/'
response = requests.get(url)
response = TextResponse(response.url, body= response.text , encoding = 'utf-8')
response

<200 https://news.daum.net/>

In [49]:
selector = '/html/body/div[2]/main/section/div/div[1]/div[1]/ul/li/div/div/strong/a/@href'
links = response.xpath(selector).extract() #data only extract 
len(links), links[:3]

(20,
 ['https://v.daum.net/v/20240923151053270',
  'https://v.daum.net/v/20240923150750118',
  'https://v.daum.net/v/20240923145528413'])

In [55]:
link = links[0]
response = requests.get(link)
response = TextResponse(response.url, body= response.text , encoding = 'utf-8')

response

<200 https://v.daum.net/v/20240923151053270>

In [61]:
title = response.xpath('//*[@id="mArticle"]/div[1]/h3/text()')[0].extract() # 
title

"전국 주택 2가구 중 1가구 이상은 '준공 20년 이상'"

### xpath
- html element 선택하는 방법
- scrapy 에서는 기본적으로 xpath를 사용
- syntax
    - // : 최상위 엘리먼트
    - \* : 모든 하위 엘리먼트 : css selector의 한칸띄우기와 같다.
    - [@id="value"] : 속성값 선택
    - / : 한단계 하위 엘리먼트 : css selector의 >와 같다.
    - [n] : nth-child(n)

## 3. items.py
- Data Model

In [65]:
# %load news/news/items.py
# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class NewsItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    title = scrapy.Field()
    link = scrapy.Field()

## 4. spider.py
- wirte crawling process

In [79]:
%%writefile news/news/spiders/spider.py

import scrapy
from news.items import NewsItem # newsitem written above


class newsSpider(scrapy.Spider):
    name = 'news'
    allow_domain = ['daum.net'] # only crawling daum site
    start_urls = ['https://news.daum.net'] # 3inital variables


    def parse(self, response):
        selector = '/html/body/div[2]/main/section/div/div[1]/div[1]/ul/li/div/div/strong/a/@href'
        links = response.xpath(selector).extract()
        for link in links:
            yield scrapy.Request(link,callback=self.parse_content)



    def parse_content(self, response):
        item = NewsItem()
        item['link'] = response.url
        item['title'] = response.xpath('//*[@id="mArticle"]/div[1]/h3/text()')[0].extract()
        yield item


##  yiled는 여러번 return 수행한다고 생각하면 편함 

Overwriting news/news/spiders/spider.py


In [2]:
def echo():
    return 1

In [4]:
echo()

1

In [6]:
def echo():
    yield 1
    yield 2
    yield 3

In [8]:
# 제너레이터 : 값 생성기
echo()

<generator object echo at 0x000002C8AF7FD6F0>

In [10]:
e =echo()

In [20]:
next(e) # 순서대로 값 나오고 ERROR 발

StopIteration: 

## 5. run scrapy
- news 디렉토리에서 아래의 커멘드 실행
- scrapy crawl news(name) -o news.csv

In [82]:
%pwd

'C:\\Users\\User\\Desktop\\에이블스쿨\\24.09\\웹크롤링\\웹크롤링_실습자료'

In [ ]:
scrapy crawl news -o news.csv